<a href="https://colab.research.google.com/github/respinar1/GoogleColaboratory/blob/main/LITAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LIT Technical Assessment

# MetaData

In [165]:
# Here I will be importing all of the necessary libraries to compute and visualize our analysis.
import pandas as pd
import math as mt
print('Done!')

Done!


# Cleaning the Data

In [2]:
# Step one: Download the CSV files and store it in a Pandas DataFrame for analysis
tbill_data = pd.read_csv('/3MO.csv')
print('Done!')

Done!


In [3]:
oil_data = pd.read_csv('/Daily Oil Prices.csv')
print('Done!')
# This CSV file gave me some trouble. I had to use Apples 'Numbers' to delete some data
# so that pandas can read the file correctly.

Done!


In [4]:
# Reviewing the Data Types
tbill_data.dtypes

DATE    object
DTB3    object
dtype: object

In [5]:
oil_data.dtypes

Day                                                   object
Cushing OK WTI Spot Price FOB  Dollars per Barrel    float64
dtype: object

In [6]:
# Notice that the DTB3 data type is object instead of float and Date/day should be in datetime.
# I will need to change that first

In [7]:
# Converts the Date/Day columns from Objects to DateTimes
tbill_data['DATE'] = pd.to_datetime(tbill_data['DATE'])

In [8]:
oil_data['Day'] = pd.to_datetime(oil_data['Day'])

In [9]:
tbill_data.dtypes

DATE    datetime64[ns]
DTB3            object
dtype: object

In [10]:
oil_data.dtypes

Day                                                  datetime64[ns]
Cushing OK WTI Spot Price FOB  Dollars per Barrel           float64
dtype: object

In [11]:
# Converts the DTB3 column from Objects to Floats
tbill_data['DTB3'].astype(float)

ValueError: ignored

In [12]:
# After reviewing the CSV file, I noticed that there were blank decimal values
# I will have to replace them with a value. (In this case, I want to change it with the average of the dataset based on month)

In [13]:
# Find all the values that do not have decimals as a value and convert those into floats
temp_data = tbill_data.loc[tbill_data['DTB3'] != "."]

In [ ]:
temp_data['DTB3'] = temp_data['DTB3'].astype(float)

In [15]:
# Using the new dataset, I can now calculate the mean based on Monthly data and replace all decimals with this value.
m_values = round(temp_data.groupby(pd.Grouper(key='DATE',freq='M')).mean(), 2)

In [ ]:
m_values['DTB3']

In [ ]:
# Based on the monthly average, if a month has a decimal, we will replace it with its monthly average.
for x in tbill_data['DATE'].loc[tbill_data['DTB3'] == "."]:
    for y in m_values['DTB3'].index.date:
        if x.strftime('%Y-%m') == y.strftime('%Y-%m'):
            tbill_data['DTB3'].loc[tbill_data['DATE'] == x] = m_values['DTB3'].loc[m_values['DTB3'].index.date == y].values[0]
            break
        else:
            continue

In [18]:
# Trying the float conversion now works fine.
tbill_data['DTB3'] = tbill_data['DTB3'].astype(float)

In [19]:
tbill_data

,DATE,DTB3
0,2016-08-26,0.33
1,2016-08-29,0.33
2,2016-08-30,0.33
3,2016-08-31,0.33
4,2016-09-01,0.33
...,...,...
1258,2021-06-24,0.05
1259,2021-06-25,0.06
1260,2021-06-28,0.05
1261,2021-06-29,0.04


In [20]:
# Now I will have to resize the data in the oil data CSV

In [ ]:
oil_data

In [22]:
# This will convert the long column into just Price

In [23]:
oil_data['Price'] = oil_data['Cushing OK WTI Spot Price FOB  Dollars per Barrel']

In [24]:
oil_data = oil_data.drop(['Cushing OK WTI Spot Price FOB  Dollars per Barrel'], axis=1)

In [25]:
oil_data

,Day,Price
0,2021-09-27,73.43
1,2021-09-24,73.43
2,2021-09-23,73.43
3,2021-09-22,72.37
4,2021-09-21,70.51
...,...,...
9003,1986-01-08,25.87
9004,1986-01-07,25.85
9005,1986-01-06,26.53
9006,1986-01-03,26.00


# Calculating Daily Excess Return

In [26]:
# Join both Tables based on their dates

In [27]:
temp_data = tbill_data.join(oil_data.set_index('Day', inplace=False), on='DATE')

In [28]:
temp_data

,DATE,DTB3,Price
0,2016-08-26,0.33,47.64
1,2016-08-29,0.33,46.97
2,2016-08-30,0.33,46.32
3,2016-08-31,0.33,44.68
4,2016-09-01,0.33,43.17
...,...,...,...
1258,2021-06-24,0.05,73.31
1259,2021-06-25,0.06,74.21
1260,2021-06-28,0.05,72.98
1261,2021-06-29,0.04,73.14


In [29]:
# Now that the data is clean I can find the difference between each value and create a percentage gain or loss
temp_data['pct change DTB3'] = temp_data['DTB3'].pct_change(periods = 1)


In [30]:
temp_data['pct change oil'] = temp_data['Price'].pct_change(periods = 1)

In [31]:
temp_data

,DATE,DTB3,Price,pct change DTB3,pct change oil
0,2016-08-26,0.33,47.64,NaN,NaN
1,2016-08-29,0.33,46.97,0.000000,-0.014064
2,2016-08-30,0.33,46.32,0.000000,-0.013839
3,2016-08-31,0.33,44.68,0.000000,-0.035406
4,2016-09-01,0.33,43.17,0.000000,-0.033796
...,...,...,...,...,...
1258,2021-06-24,0.05,73.31,0.000000,0.002736
1259,2021-06-25,0.06,74.21,0.200000,0.012277
1260,2021-06-28,0.05,72.98,-0.166667,-0.016575
1261,2021-06-29,0.04,73.14,-0.200000,0.002192


In [32]:
# Drops all the NaN 
temp_data = temp_data.dropna()

In [ ]:
# Calculating the Excess Return
temp_data['Daily Excess Return'] = temp_data['pct change DTB3'] - temp_data['pct change oil']

In [35]:
temp_data

,DATE,DTB3,Price,pct change DTB3,pct change oil,Daily Excess Return
1,2016-08-29,0.33,46.97,0.000000,-0.014064,0.014064
2,2016-08-30,0.33,46.32,0.000000,-0.013839,0.013839
3,2016-08-31,0.33,44.68,0.000000,-0.035406,0.035406
4,2016-09-01,0.33,43.17,0.000000,-0.033796,0.033796
5,2016-09-02,0.32,44.39,-0.030303,0.028260,-0.058563
...,...,...,...,...,...,...
1258,2021-06-24,0.05,73.31,0.000000,0.002736,-0.002736
1259,2021-06-25,0.06,74.21,0.200000,0.012277,0.187723
1260,2021-06-28,0.05,72.98,-0.166667,-0.016575,-0.150092
1261,2021-06-29,0.04,73.14,-0.200000,0.002192,-0.202192


# Calculating Log Cumulative Excess Return

In [153]:
# I will make a new 'temp' table to calculate the Log Excess Return
temp_data = tbill_data.join(oil_data.set_index('Day', inplace=False), on='DATE')

In [154]:
temp_data

,DATE,DTB3,Price
0,2016-08-26,0.33,47.64
1,2016-08-29,0.33,46.97
2,2016-08-30,0.33,46.32
3,2016-08-31,0.33,44.68
4,2016-09-01,0.33,43.17
...,...,...,...
1258,2021-06-24,0.05,73.31
1259,2021-06-25,0.06,74.21
1260,2021-06-28,0.05,72.98
1261,2021-06-29,0.04,73.14


In [148]:
# Finding the Log Cumulative is fairly simple; by taking the last value of DTB3 and the first value of DTB3
# then subtracting the Log Cumulative of Oil Price to get the Log Cumulative Excess Return

In [162]:
x = mt.log(temp_data['DTB3'].iloc[-1]/temp_data['DTB3'].iloc[0])

In [163]:
y = mt.log(temp_data['Price'].iloc[-1]/temp_data['Price'].iloc[0])

In [164]:
print(x-y)

-2.320954382592712
